<a href="https://colab.research.google.com/github/ersjunior/3DTS-MachineLearningOps/blob/main/Aula_2/Concess%C3%A3o_de_Cr%C3%A9dito_com_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [3]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 8  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "52.152.221.137" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}

#@markdown Informe o seu nome:
students_name = "Eliezer Ramos Silveira Junior" #@param {type:"string"}

#@markdown Ah, e não se esqueça de executar esta célula!


In [4]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [5]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

In [6]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
27205,"Jeremy, Vigil",2087.111178,28.661946,1,0,0,0,0,1
42445,"Che'Lisa, Martinez",3879.911093,52.684871,0,1,0,0,0,2
21438,"Andre, Bunting",3201.924080,59.942462,0,0,0,1,1,1
32636,"Samuel, Anthony",2939.629678,46.641180,0,0,1,1,1,3
45547,"Emoni, Gadison",3997.646300,30.393005,0,1,0,0,0,3
2297,"Dwayne, Hall",5379.520504,55.929759,0,0,1,1,0,0
40130,"Shannon, Park",2356.156358,49.237147,1,1,0,0,0,1
19406,"Frederico, Torres",4438.851024,36.403576,0,0,0,1,0,2


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [7]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [8]:
response01 = requests.request("POST", url + f"?model=modelo01&aluno=a", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [1, 1, 0, 0, 1, 0, 1, 0],
 'proba': [[0.0005345257699967965, 0.9994654742300032],
  [0.3431606287280107, 0.6568393712719893],
  [0.8219702357479396, 0.17802976425206044],
  [0.964287033034834, 0.03571296696516598],
  [0.39254471821053727, 0.6074552817894627],
  [0.9993267443090346, 0.0006732556909654235],
  [0.0006816213409724936, 0.9993183786590275],
  [0.9694402820418832, 0.030559717958116808]]}

In [9]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [10]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [1.0209591258105453,
  0.7518969236497837,
  0.4715457262646926,
  0.062033713111870184,
  0.7059686699321438,
  -0.009885484195572516,
  0.9951210154753222,
  0.03735721868228562]}

In [11]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [12]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
27205,"Jeremy, Vigil",1,"[0.0005345257699967965, 0.9994654742300032]",1.020959
42445,"Che'Lisa, Martinez",1,"[0.3431606287280107, 0.6568393712719893]",0.751897
21438,"Andre, Bunting",0,"[0.8219702357479396, 0.17802976425206044]",0.471546
32636,"Samuel, Anthony",0,"[0.964287033034834, 0.03571296696516598]",0.062034
45547,"Emoni, Gadison",1,"[0.39254471821053727, 0.6074552817894627]",0.705969
2297,"Dwayne, Hall",0,"[0.9993267443090346, 0.0006732556909654235]",-0.009885
40130,"Shannon, Park",1,"[0.0006816213409724936, 0.9993183786590275]",0.995121
19406,"Frederico, Torres",0,"[0.9694402820418832, 0.030559717958116808]",0.037357
